In [1]:
import pickle
import nltk
import tensorflow as tf
import numpy as np
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import pandas as pd
import re
from keras.preprocessing.sequence import pad_sequences
import os
!pip install gradio
import gradio as gr

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('/content/drive/MyDrive/Models/Anxiety_Dict.p', 'rb') as handle:
    anxiety_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Control_Dict.p', 'rb') as handle:
    control_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Depression_Dict.p', 'rb') as handle:
    depression_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Autism_Dict.p', 'rb') as handle:
    autism_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/BPD_Dict.p', 'rb') as handle:
    BPD_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Bipolar_Dict.p', 'rb') as handle:
    bipolar_word_index = pickle.load(handle)

with open('/content/drive/MyDrive/Models/Schizo_Dict.p', 'rb') as handle:
    schizo_word_index = pickle.load(handle)


anxiety_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Anxiety')
control_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Control')
depression_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Depression')
autism_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Autism')
BPD_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_BPD')
bipolar_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Bipolar')
schizo_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/MHSMC_Model_Schizo')

In [3]:
def detokenize(tokens):
    text = ' '.join(tokens)
    return text

def preprocess_corpus(tokens):
  tokens = [t.lower() for t in tokens] # make lowercase

  stop = set(stopwords.words('english'))
  tokens = [t for t in tokens if t not in stop] # remove stop words
  tokens = list(filter(None, tokens)) # get rid of extra spaces

  text = detokenize(tokens)


  #text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text) #put spaces between word and special characters
  #text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text) #put spaces between special characters and word
  #text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"([.,;:!?'\"“\(])(\w)", text) #put spaces between special characters and other special characters

  pattern = r"[{}]".format(string.punctuation) 
  text = re.sub(pattern, "", text)  # remove puntucation 
  text = re.sub(r'\b\d+\b', '', text) # only accept text
  
  tokens = re.split(r"\s+",text) #split the words into array
  tokens = list(filter(None, tokens)) # get rid of extra spaces
  
  return tokens

def getProcessCorpus(data):
  processed_data = []
  for i in range(len(data)):
    sentence = data[i]
    p_sentence = preprocess_corpus(sentence)
    processed_data.append(p_sentence[1:len(p_sentence)])
  return processed_data

In [16]:
def runMentalHealthAlarmSystem(input_sentence):
  processed_sentence = preprocess_corpus(input_sentence.split(' '))

  x_anxiety = []
  x_control = []
  x_depression = []
  x_autism = []
  x_BPD = []
  x_bipolar = []
  x_schizo = []

  x_anxiety_numeric = []
  x_control_numeric = []
  x_depression_numeric = []
  x_autism_numeric = []
  x_BPD_numeric = []
  x_bipolar_numeric = []
  x_schizo_numeric = []

  for token in processed_sentence:
    x_anxiety_numeric.append(anxiety_word_index[token])
    x_control_numeric.append(control_word_index[token])
    x_depression_numeric.append(depression_word_index[token])
    x_autism_numeric.append(autism_word_index[token])
    x_BPD_numeric.append(BPD_word_index[token])
    x_bipolar_numeric.append(bipolar_word_index[token])
    x_schizo_numeric.append(schizo_word_index[token])

  x_anxiety.append(x_anxiety_numeric)
  x_control.append(x_control_numeric)
  x_depression.append(x_depression_numeric)
  x_autism.append(x_autism_numeric)
  x_BPD.append(x_BPD_numeric)
  x_bipolar.append(x_bipolar_numeric)
  x_schizo.append(x_schizo_numeric)

  max_sequence_length = 5000

  x_anxiety = np.array(pad_sequences(x_anxiety, maxlen = 4792, padding = 'pre'))
  x_control = np.array(pad_sequences(x_control, maxlen = 4792, padding = 'pre'))
  x_depression = np.array(pad_sequences(x_depression, maxlen = 8403, padding = 'pre'))
  x_autism = np.array(pad_sequences(x_autism, maxlen = 3902, padding = 'pre'))
  x_BPD = np.array(pad_sequences(x_BPD, maxlen = 2996, padding = 'pre'))
  x_bipolar = np.array(pad_sequences(x_bipolar, maxlen = 4792, padding = 'pre'))
  x_schizo = np.array(pad_sequences(x_schizo, maxlen = 2781, padding = 'pre'))

  mental_health_stats = []

  anxiety_percentage = (anxiety_model.predict(x_anxiety))
  control_percentage = (control_model.predict(x_control))
  depression_percentage = (depression_model.predict(x_depression))
  autism_percentage = (autism_model.predict(x_autism))
  BPD_percentage = (BPD_model.predict(x_BPD))
  bipolar_percentage = (bipolar_model.predict(x_bipolar))
  schizo_percentage = (schizo_model.predict(x_schizo))

  mental_health_stats.append("{:.2%}".format(anxiety_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(control_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(depression_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(autism_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(BPD_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(bipolar_percentage[0][1]))
  mental_health_stats.append("{:.2%}".format(schizo_percentage[0][1]))

  df = pd.DataFrame(mental_health_stats, columns = ['Percentage Chance of Disorder'])
  df['Disorder'] = ['Anxiety', 'Control', 'Depression', 'Autism', 'Borderline Personality', 'Bipolar', 'Schizo']
  df['Description'] = ['Assess similarity to Reddit discussion of Anxiety', 
                       'Assess similarity to Reddit forum to discuss, vent, support and share information about mental health, illness, and wellness.', 
                       'Assess similarity to Reddit discussion of Depression',
                       'Assess similarity to Reddit discussion of Autism',
                       'Assess similarity to Reddit discussion of Borderline Personality Disorder',
                       'Assess similarity to Reddit discussion of Bipolar',
                       'Assess similarity to Reddit disccusion of Schizo']
  
  return df


In [17]:
sentence = 'I am depressed and upset, and I dont know what to do'
df = runMentalHealthAlarmSystem(sentence)
print(df)

  Percentage Chance of Disorder  ...                                        Description
0                         0.72%  ...  Assess similarity to Reddit discussion of Anxiety
1                        34.18%  ...  Assess similarity to Reddit forum to discuss, ...
2                        96.03%  ...  Assess similarity to Reddit discussion of Depr...
3                        38.39%  ...   Assess similarity to Reddit discussion of Autism
4                        28.15%  ...  Assess similarity to Reddit discussion of Bord...
5                        39.26%  ...  Assess similarity to Reddit discussion of Bipolar
6                         2.83%  ...   Assess similarity to Reddit disccusion of Schizo

[7 rows x 3 columns]


In [18]:
iface = gr.Interface(fn=runMentalHealthAlarmSystem, inputs="text", outputs="dataframe")
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://28505.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7864/',
 'https://28505.gradio.app')